In [ ]:
import numpy as np

scores = np.load('results_f1.npy')
print("\nScores:\n", scores.shape)


In [ ]:
mean_scores = np.mean(scores, axis=2).T
print("\nMean scores:\n", mean_scores)

In [ ]:
from sklearn.svm import SVC
#Zaimplementowane metody
from StratifiedBagging import StratifiedBagging
from implementedBoosting import implementedBoosting
from RandomSubspace import RandomSubspace
from RandomPatches import RandomPatches

 clfs = {
     'SVC' : SVC(),
     'sBag': StratifiedBagging(),    
     'BST': implementedBoosting(),
     'RSP' : RandomSubspace(),
     'RPT' : RandomPatches(),
 }

In [ ]:
from scipy.stats import rankdata
ranks = []
for ms in mean_scores:
    ranks.append(rankdata(ms).tolist())
ranks = np.array(ranks)
print("\nRanks:\n", ranks)

In [ ]:
mean_ranks = np.mean(ranks, axis=0)
print("\nMean ranks:\n", mean_ranks)

In [ ]:
from scipy.stats import ranksums

alfa = .05
w_statistic = np.zeros((len(clfs), len(clfs)))
p_value = np.zeros((len(clfs), len(clfs)))

for i in range(len(clfs)):
    for j in range(len(clfs)):
        w_statistic[i, j], p_value[i, j] = ranksums(ranks.T[i], ranks.T[j])

In [ ]:
from tabulate import tabulate

headers = list(clfs.keys())
names_column = np.expand_dims(np.array(list(clfs.keys())), axis=1)
w_statistic_table = np.concatenate((names_column, w_statistic), axis=1)
w_statistic_table = tabulate(w_statistic_table, headers, floatfmt=".2f")
p_value_table = np.concatenate((names_column, p_value), axis=1)
p_value_table = tabulate(p_value_table, headers, floatfmt=".2f")
print("\nw-statistic:\n", w_statistic_table, "\n\np-value:\n", p_value_table)

In [ ]:
advantage = np.zeros((len(clfs), len(clfs)))
advantage[w_statistic > 0] = 1
advantage_table = tabulate(np.concatenate(
    (names_column, advantage), axis=1), headers)
print("\nAdvantage:\n", advantage_table)

In [ ]:
significance = np.zeros((len(clfs), len(clfs)))
significance[p_value <= alfa] = 1
significance_table = tabulate(np.concatenate(
    (names_column, significance), axis=1), headers)
print("\nStatistical significance (alpha = 0.05):\n", significance_table)

In [ ]:
stat_better = significance * advantage
stat_better_table = tabulate(np.concatenate(
    (names_column, stat_better), axis=1), headers)
print("Statistically significantly better:\n", stat_better_table)